In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
path = '/content/drive/MyDrive/kw-ai'

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
warnings.filterwarnings(action='ignore')

import os
from sklearn.preprocessing import StandardScaler
from keras import backend as K
from sklearn.model_selection import KFold

In [34]:
# data load
# path 변수를 적절히 변경
x_train_path = os.path.join(path, 'data/train_features.csv')
y_train_path = os.path.join(path, 'data/train_labels.csv')
x_test_path = os.path.join(path, 'data/test_features.csv')
sub_path = os.path.join(path, 'data/sample_submission.csv')

train = pd.read_csv(x_train_path)
train_label = pd.read_csv(y_train_path)
test = pd.read_csv(x_test_path)
sub = pd.read_csv(sub_path)

In [35]:
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

In [36]:
dt=0.02 
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [37]:
train_dt=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)

100%|██████████| 3125/3125 [01:00<00:00, 51.81it/s]


In [38]:
test_dt=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|██████████| 782/782 [00:13<00:00, 57.07it/s]


In [39]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [40]:
train=pd.concat(train_dt)

In [41]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for i in train.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|██████████| 3125/3125 [00:13<00:00, 224.75it/s]


In [42]:
test=pd.concat(test_dt)

In [43]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|██████████| 782/782 [00:02<00:00, 334.74it/s]


In [44]:
col=train.columns
train_s=train.copy()
test_s=test.copy()

In [45]:
scaler = StandardScaler()

train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
train_sc = pd.DataFrame(data = train_s,columns =col)

test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
test_sc = pd.DataFrame(data = test_s,columns =col)

In [46]:
# make dataset
# x를 시계열 데이터로 변경 시켜주는 함수
def make_series(data):
  ids = data['id'].unique()
  id_data = data.groupby('id')
  series_data = []

  for i in ids:
    df = id_data.get_group(i)
    df = df.drop(['id', 'time'], axis=1)
    series_data.append(df.to_numpy())

  series_data = np.array(series_data)
  return series_data


def ids_26(label):
    mask = label['label'] == 26
    ids = label.loc[mask, 'id'].tolist()
    
    return ids


def make_split_dataset(series_train, ids, labels):
    final_list = []
    
    k_split = KFold(n_splits=5)
    
    temp = np.array(range(3125))
    except_train_mask = np.setdiff1d(temp, ids, assume_unique=True)
    except_train = series_train[except_train_mask]
    except_label = labels[except_train_mask]
    
    train_26 = series_train[ids]

    for _, fold in k_split.split(train_26):
        temp_train = train_26[fold]
        temp_label = np.array([26] * len(temp_train))
        
        temp_train = np.concatenate([temp_train, except_train], axis=0)
        temp_label = np.concatenate([temp_label, except_label], axis=0)
        print(temp_train.shape)
        final_list.append([temp_train, temp_label])
        
    return final_list

In [47]:
series_train = make_series(train_sc)
series_test = make_series(test_sc)

In [48]:
ids = ids_26(train_label)
data_list = make_split_dataset(series_train, ids, train_label['label'])

(1911, 600, 18)
(1911, 600, 18)
(1911, 600, 18)
(1910, 600, 18)
(1910, 600, 18)


In [49]:
import tensorflow as tf

In [50]:
# dataset과 validation set을 만들어 주는 함수
# validation set은 shuffle 적용 x
def make_train(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 128
  train_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000, seed=42)
  train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

def make_val(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 128
  val_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  val_dataset = val_dataset.batch(BATCH_SIZE)
  val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

In [51]:
from tensorflow import keras

In [52]:
# 모델을 만들어 주는 함수
# 기존 base에서 overfitting이 심해, dropout을 늘림(아직 제출은 안해봄)
def base():
  model = keras.models.Sequential([
            keras.layers.Conv1D(128, 9, padding='same', input_shape=[600, 18]),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.3),
            keras.layers.Conv1D(256, 6, padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.4),
            keras.layers.Conv1D(128, 3,padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.5),
            keras.layers.GRU(256, return_sequences=True, dropout=0.2),
            keras.layers.MaxPool1D(),
            keras.layers.GRU(150, return_sequences=True, dropout=0.2),
            keras.layers.GlobalAveragePooling1D(),
            keras.layers.Dense(61, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
  return model

In [53]:
# checkpoint path
# 중간중간 모델의 weight를 저장할 경로 설정
ckpt_name = 'random_sampling.hdf5'
checkpoint_dir_path = os.path.join(path, 'checkpoint')
checkpoint_path = os.path.join(path, 'checkpoint', ckpt_name)

# check checkpoint paht
# 경로가 없으면 생성함
if not(os.path.exists(checkpoint_dir_path)):
  os.mkdir(checkpoint_dir_path)

# callback 함수 목록
callbacks_list = [
    # 매 epoch 마다 val_loss를 체크하여 가장 낮은 상태의 weight를 저장
    tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_path,
        monitor='loss',
        mode='min',
        save_weights_only=True,
        save_best_only=True
    ),
    # 8번 동안 val_loss의 향상이 없으면 훈련 종료
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        mode='min',
        verbose=1, 
        patience=8
    ) 
]

In [54]:
import time

In [55]:
models = []
for series in data_list:
  train_dataset = make_train(series[0], series[1])

  model = base()

  model.fit(train_dataset, epochs=100)
  #model.load_weights(checkpoint_path)
  model.evaluate(train_dataset)

  models.append(model)

Epoch 1/100
15/15 [==============================] - 12s 433ms/step - loss: 3.7587 - accuracy: 0.0842
Epoch 2/100
15/15 [==============================] - 7s 431ms/step - loss: 3.0551 - accuracy: 0.2344
Epoch 3/100
15/15 [==============================] - 7s 432ms/step - loss: 2.8127 - accuracy: 0.2585
Epoch 4/100
15/15 [==============================] - 7s 433ms/step - loss: 2.7088 - accuracy: 0.3150
Epoch 5/100
15/15 [==============================] - 7s 435ms/step - loss: 2.4066 - accuracy: 0.3464
Epoch 6/100
15/15 [==============================] - 7s 432ms/step - loss: 2.3107 - accuracy: 0.3511
Epoch 7/100
15/15 [==============================] - 7s 431ms/step - loss: 2.1732 - accuracy: 0.4040
Epoch 8/100
15/15 [==============================] - 7s 433ms/step - loss: 2.0374 - accuracy: 0.4338
Epoch 9/100
15/15 [==============================] - 7s 431ms/step - loss: 1.8867 - accuracy: 0.4715
Epoch 10/100
15/15 [==============================] - 7s 432ms/step - loss: 1.7936 - accur

KeyboardInterrupt: ignored

In [56]:
model.evaluate(train_dataset)

15/15 [==============================] - 3s 141ms/step - loss: 0.8987 - accuracy: 0.7891


[0.8987201452255249, 0.7891156673431396]

In [ ]:
for model in models:
  model.evaluate(train_dataset)

In [ ]:
# 결과 생성
pred_list = []    # 예측 결과를 담을 리스트
for model in models:
  pred = model.predict(series_test)
  pred_list.append(pred)

pred = np.mean(pred_list, axis=0)

In [ ]:
# 제출물 생성
sub.iloc[:, 1:] = pred
sub.to_csv('random_sampling.csv', index=False)